In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
%cd ..

/cis/home/dpacker/my_documents/equivariant-attention


In [2]:
from models import rotation_attention, positional_encodings, equivariance
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from matplotlib import colormaps, animation
rng = jax.random.PRNGKey(seed=1)
rngs = jax.random.split(rng, 10)


In [4]:
n_samples = 10
n_keys = 5
n_group_samples = 15
d = 2

X_train = positional_encodings.uniform_ball_samples(rngs[0], n_samples, d = d, r = 1.)
X_train_encoded = positional_encodings.positional_encoding(X_train, method = "stereographic")

X_test = positional_encodings.uniform_ball_samples(rngs[1], n_samples, d = d, r = 1.)
X_test_encoded = positional_encodings.positional_encoding(X_test, method = "stereographic")

rotation = rotation_attention.group_samples(360, 0)[113]
f = lambda x : rotation @ x * jnp.linalg.norm(x)

Y_train = jax.vmap(f)(X_train)
noise = jax.random.normal(rngs[2], shape=Y_train.shape) * 0.1
Y_test = jax.vmap(f)(X_test)

stereographic_encoding = lambda v: positional_encodings.positional_encoding(
    v, method="stereographic"
)

In [6]:

n_epochs = 5000

keys_hist, values_hist, key_reps, value_reps, betas_hist = rotation_attention.train(
    rngs[3],
    X_train,
    stereographic_encoding,
    n_keys,
    n_group_samples,
    n_epochs=n_epochs,
    lr=1.0,
    Y_train=Y_train + noise, 
    verbose=True,
    # init_values=init_values,
    # init_keys=init_keys,
    init_beta = 1.
)

  0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
func = lambda v: rotation_attention.call_fn(v, keys_hist[-1], key_reps, values_hist[-1], value_reps, betas_hist[-1])

domain_reps = rotation_attention.group_samples(1_000, 1)
codomain_reps = rotation_attention.group_samples(1_000, 0)

equivariance.batched_check_equivariance(func, stereographic_encoding(X_test), domain_reps, codomain_reps)